In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/NLP/train.csv'

In [3]:
import pandas as pd
data = pd.read_csv(path)

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
###Drop Nan Values
data=data.dropna()

In [7]:
## Get the Independent Features
X=data.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=data['label']

In [9]:
# Import libraries

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [10]:
# copy data
messages=X.copy()

In [12]:
# messages reset
messages.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

**One Hot Encoding**

In [17]:
voc_size=5000

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1034, 2017, 1528, 3652, 2526, 457, 3391, 599, 1589, 3570],
 [4807, 3179, 743, 134, 2380, 1924, 4865],
 [3610, 2949, 3938, 3554],
 [836, 4926, 3405, 3431, 141, 4809],
 [1847, 2380, 838, 2373, 3848, 1761, 2380, 4950, 3244, 1887],
 [1694,
  2148,
  2935,
  4804,
  2277,
  1554,
  4067,
  3742,
  618,
  1483,
  4873,
  1217,
  3308,
  3240,
  4865],
 [2286, 197, 900, 4029, 172, 2381, 4693, 2902, 2727, 3792, 2238],
 [3491, 3292, 2514, 640, 1926, 3988, 1554, 1196, 2727, 3792, 2238],
 [1693, 2932, 2097, 3169, 1084, 1562, 1726, 4998, 1554, 2660],
 [1994, 2962, 872, 2079, 4540, 1056, 2700, 4839],
 [543, 4020, 3234, 1386, 4099, 2322, 1649, 3788, 2753, 1311, 4133],
 [3431, 4466, 2526, 1562, 1554, 1926],
 [1399, 1063, 3099, 2761, 4544, 4217, 4991, 2850, 1404],
 [1680, 782, 819, 4970, 3628, 3518, 995, 2727, 3792, 2238],
 [4317, 2639, 4197, 3930, 3625, 2727, 3792, 2238],
 [1254, 668, 2828, 2223, 4929, 3162, 3947, 2076, 3899, 3294],
 [1815, 45, 3179],
 [3305, 3167, 2164, 4091, 1554, 2956, 631, 4865

**Word Embedding representation**

In [19]:
# Padding sequences
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  599 1589 3570]
 [   0    0    0 ... 2380 1924 4865]
 [   0    0    0 ... 2949 3938 3554]
 ...
 [   0    0    0 ... 2727 3792 2238]
 [   0    0    0 ...  527 4231 4367]
 [   0    0    0 ... 4037 1786 3323]]


In [20]:

## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 42ms/step - loss: 0.3085 - accuracy: 0.8516 - val_loss: 0.2025 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1431 - accuracy: 0.9476 - val_loss: 0.1945 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0986 - accuracy: 0.9663 - val_loss: 0.2154 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0720 - accuracy: 0.9756 - val_loss: 0.2584 - val_accuracy: 0.9171
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0464 - accuracy: 0.9852 - val_loss: 0.3029 - val_accuracy: 0.9162
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0355 - accuracy: 0.9883 - val_loss: 0.4094 - val_accuracy: 0.9090
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0267 - accuracy: 0.9913 - val_loss: 0.4350 - val_accuracy: 0.9067

**Performance metrics**

In [24]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3134,  285],
       [ 303, 2313]])

In [27]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9025683512841757

In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

